In this file is the implementation of the new decay chain files with: 
* How a chain is represented in the code.
* How a chain is created and written to yaml-file. 
* How a chain is printed in markdown. 

Structure of previous decay chain input file:

In [34]:
!cat UF_C++/E115_chains/chain_example.txt 
print("Next")
!cat UF_C++/E115_chains/chain1403.txt 

1301    //ID
242.1   //Beam energy
465     //Pixel or similar
0       //Extra
0       //Extra
13.4    //Implantation energy
0       //Implantation time == 0
10.29   //First energy
10      //Energy uncertainty
0.366   //First time
9.24    //Second energy
10      //Energy uncertainty
0.935   //Second time
9.77    //Third energy
10
11.1
0.825
30
0.515
9.06
10
15.3
197
0       //Fission uncertainty == 0
17.4
Next
1403
242.1
681
0
0
13.9
0
10.54
40
0.261
9.95
50
1.15
196
0
0.343


In [3]:
import numpy as np
import yaml
import sys
from collections import OrderedDict

In [6]:
def dump_anydict_as_map(anydict):
    yaml.add_representer(anydict, _represent_dictorder)

def _represent_dictorder(self, data):
    if isinstance(data, DecayChain):
        return self.represent_mapping('tag:yaml.org,2002:map', data.__getstate__().items())
    else:
        return self.represent_mapping('tag:yaml.org,2002:map', data.items())

In [2]:
d = {'a':1, 'b':2, 'c':3}
yaml.dump(d)

'{a: 1, b: 2, c: 3}\n'

## pyyaml

<https://pyyaml.org/wiki/PyYAMLDocumentation>

In [4]:
class DecayChain(yaml.YAMLObject):

    yaml_tag = u'!DecayChain'
    
    def __init__(self, idd="", be="", pi="", ie="", it="", ea=[], eas=[], ta=[]):
        self.ID = idd
        self.BeamEnergy = be
        self.Pixel = pi
        self.ImplantEnergy = ie
        self.ImplantTime = it
        self.EAlpha, self.EAlphaSigma, self.TAlpha = ea, eas, ta
        represent_dict_order = lambda self, data:  self.represent_mapping('tag:yaml.org,2002:map', data.items())
        yaml.add_representer(OrderedDict, represent_dict_order)
        
    def __repr__(self):
        return "%s(ID=%r, BeamEnergy=%r, Pixel=%r, ImplantEnergy=%r)" (self.__class__.__name__, self.ID, self.BeamEnergy, self.Pixel, self.ImplantEnergy)
    
    def __getstate__(self):
        d = OrderedDict()
        d['ID'] = self.ID
        d['BeamEnergy'] = self.BeamEnergy
        d['Pixel'] = self.Pixel
        d['ImplantEnergy'] = self.ImplantEnergy
        d['ImplantTime'] = self.ImplantTime
        d['EAlpha'] = self.EAlpha
        d['EAlphaSigma'] = self.EAlphaSigma
        d['TAlpha'] = self.TAlpha        
        return d
        
    def ReadUFFile(self, s_ex):
        #s_ex = "UF_C++/E115_chains/chain1403.txt" #test file
        with open(s_ex, "r") as f:
            lines = f.readlines()
            for i, l in enumerate(lines):
                if l.strip():
                    lines[i] = l.split()[0]
                    if lines[i] == '\n':
                        del lines[i]
                else:
                    if i < 9:
                        print("Found weird line too early! Exiting.")
                        sys.exit(1)
                    lines = lines[0:i] 
            self.ID = lines[0]
            self.BeamEnergy = float(lines[1])
            self.Pixel = int(lines[2])
            self.ImplantEnergy = float(lines[5])
            self.ImplantTime = float(lines[6])
            self.EAlpha, self.EAlphaSigma, self.TAlpha = [], [], []
            for l in lines[7::3]:
                self.EAlpha.append(float(l))
            for l in lines[8::3]:
                self.EAlphaSigma.append(float(l))
            for l in lines[9::3]:
                self.TAlpha.append(float(l))
            
                
        #self.EAlpha = np.array(self.EAlpha, dtype=float)
        #self.EAlphaSigma = np.array(self.EAlphaSigma, dtype=float)
        #self.TAlpha = np.array(self.TAlpha, dtype=float)
        
c = DecayChain()
c.ReadUFFile("UF_C++/E115_chains/chain1403.txt")
print("EAlpha=", c.EAlpha)
print("EAlphaSigma=", c.EAlphaSigma)
print("TAlpha=", c.TAlpha)

print(yaml.dump(c, sys.stdout))

EAlpha= [10.54, 9.95, 196.0]
EAlphaSigma= [40.0, 50.0, 0.0]
TAlpha= [0.261, 1.15, 0.343]
!DecayChain
BeamEnergy: 242.1
EAlpha: [10.54, 9.95, 196.0]
EAlphaSigma: [40.0, 50.0, 0.0]
ID: '1403'
ImplantEnergy: 13.9
ImplantTime: 0.0
Pixel: 681
TAlpha: [0.261, 1.15, 0.343]
None


Dumping file after file transformation

In [7]:
s_out = "data/"
s_file = s_out+c.ID+"Chain.yml"
f_out = open(s_file, "w")
yaml.dump(c, f_out)
#https://stackoverflow.com/questions/16782112/can-pyyaml-dump-dict-items-in-non-alphabetical-order
f_out.close()

In [13]:
f_in = open(s_file, "r")
print(f_in.read())

!DecayChain
BeamEnergy: 242.1
EAlpha: [10.54, 9.95, 196.0]
EAlphaSigma: [40.0, 50.0, 0.0]
ID: '1403'
ImplantEnergy: 13.9
ImplantTime: 0.0
Pixel: 681
TAlpha: [0.261, 1.15, 0.343]



Reading in dumped file into `DecayChain` object.

In [21]:
f_in = open(s_file, 'r')
c2 = yaml.load(f_in)
print(yaml.dump(c2))
f_in.close()

!DecayChain
BeamEnergy: 242.1
EAlpha: [10.54, 9.95, 196.0]
EAlphaSigma: [40.0, 50.0, 0.0]
ID: '1403'
ImplantEnergy: 13.9
ImplantTime: 0.0
Pixel: 681
TAlpha: [0.261, 1.15, 0.343]



## Generating new files

In [6]:
s_in = "UF_C++/E117_chains/"
import os
files = os.listdir(s_in)
print(files)

['117_chain_T01.txt', 'E117_chain2006.txt', 'chain2102.txt~', 'E117_chain2006.txt~', 'E117_chain2013.txt~', 'E117_chain2010.txt', 'E117_chain2002.txt', 'E117_10_short_full_chains', 'E117_chain2104.txt', 'E117_chain2104.txt~', 'chain2105.txt~', '117_chain_T02.txt', 'E117_chain2012.txt', 'E117_chain2105.txt', 'E117_chain2005.txt~', 'E117_chain2007.txt', 'chain2106.txt~', 'E117_chain2001.txt', '117_chain_L02.txt', 'E117_chain2014.txt', 'E117_chain2016.txt', 'E117_chain2011.txt', 'E117_chain2010.txt~', 'E117_chain2002.txt~', 'chain2101.txt~', 'chain2103.txt~', 'E117_chain2003.txt~', '117_chain_L01.txt', 'E117_chain2003.txt', 'E117_chain2106.txt', 'old_chains', 'E117_chain2102.txt', 'E117_chain2001.txt~', 'E117_chain2013.txt', '#117_chain_L01.txt#', '117_chain_L03.txt', 'E117_chain2005.txt', 'E117_chain2004.txt~', '#117_chain_S05.txt#', 'E117_chain2103.txt', '117_chain_L04.txt', 'E117_chain2008.txt', 'E117_chain2101.txt', 'E117_chain2004.txt', 'E117_chain2015.txt', 'E117_chain2009.txt', 'ch

# Here UFs format is transformed to the yaml files

In [8]:
import re

s_files = " ".join(files)
files = re.findall(string=s_files, pattern="E117_chain\d+.txt")

c = DecayChain()

for f in files:
    print("File:", f)
    try:
        c.ReadUFFile(s_in+f)
        s_file = s_out+"Chain"+c.ID+".yml"
        f_out = open(s_file, "w")        
        yaml.dump(c, f_out)
        f_out.close()
    except IndexError:
        print("Index error caught, printing file content")
        with open(s_in+f, 'r') as fin:
            lines = fin.readlines()
            for i, l in enumerate(lines):
                print(i, l)
                

File: E117_chain2006.txt
File: E117_chain2006.txt
File: E117_chain2013.txt
File: E117_chain2010.txt
File: E117_chain2002.txt
File: E117_chain2104.txt
File: E117_chain2104.txt
File: E117_chain2012.txt
File: E117_chain2105.txt
File: E117_chain2005.txt
File: E117_chain2007.txt
File: E117_chain2001.txt
File: E117_chain2014.txt
File: E117_chain2016.txt
File: E117_chain2011.txt
File: E117_chain2010.txt
File: E117_chain2002.txt
File: E117_chain2003.txt
File: E117_chain2003.txt
File: E117_chain2106.txt
File: E117_chain2102.txt
File: E117_chain2001.txt
File: E117_chain2013.txt
File: E117_chain2005.txt
File: E117_chain2004.txt
File: E117_chain2103.txt
File: E117_chain2008.txt
File: E117_chain2101.txt
File: E117_chain2004.txt
File: E117_chain2015.txt
File: E117_chain2009.txt


In [ ]:
%%add_to Chain #https://alexhagen.github.io/jdc/

In [ ]:
class Monster(yaml.YAMLObject):
    yaml_tag = u'!Monster'
    def __init__(self, name, hp, ac, attacks):
        self.name = name
        self.hp = hp
        self.ac = ac
        self.attacks = attacks
    #def __repr__(self):
     #   return "%s(name=%r, hp=%r, ac=%r, attacks=%r)" % (
      #       self.__class__.__name__, self.name, self.hp, self.ac, self.attacks)
    


In [ ]:
c = yaml.load("""
... --- !Monster
... name: Cave spider
... hp: [2,6]    # 2d6
... ac: 16
... attacks: [BITE, HURT]
... """)
print(c.attacks)

In [ ]:
print(yaml.dump(Monster(
...     name='Cave lizard', hp=[3,6], ac=16, attacks=['BITE','HURT'])))